# **0. Inisiasi Google Colab**

## install module

In [ ]:
!pip install basemap
# !pip install basemap-data-hires

In [ ]:
!pip install obspy

## inisiasi colab notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd drive/MyDrive/colab

In [ ]:
%pwd

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import datetime
import plotly.express as px
from mpl_toolkits.basemap import Basemap
import obspy

# **1. Konversi Data ke Format STEAD**

In [ ]:
from convertArrival import arrival2stead
arrival2stead(arrival_fname='arrival.txt',
              station_fname='indonesia_station.txt')

In [ ]:
df = pd.read_csv("arrival.csv")

In [ ]:
df.source_origin_time

# **2. Analisis Awal Data**

## Cek Isi Dataset

In [ ]:
print(f'Jumlah baris pada dataset: {df.shape[0]}')
print(f'Jumlah fitur (kolom) pada dataset: {df.shape[1]}')
print(f'Jumlah duplikasi pada dataset: {df.duplicated().sum()}')
print(f'Jumlah nilai kosong pada dataset: {sum(df.isna().sum())}')

In [ ]:
df.isnull().sum()[df.isnull().sum() > 0]

In [ ]:
df.columns

## Jumlah Gempabumi

In [ ]:
df.source_id.unique()

In [ ]:
df_unique = df.drop_duplicates(subset=['source_id'])

In [ ]:
print(f"jumlah gempa: {len(df_unique)}")

## Distribusi Kedalaman

In [ ]:
plt.hist(df_unique['source_depth_km'],
         bins = [x for x in range(0,600,50)])

## Distribusi Magnitudo

In [ ]:
df_unique.source_magnitude.describe()

In [ ]:
fig = ff.create_distplot([df_unique['source_magnitude']], ['Earthquake Magnitude'])
fig.show()

## Distribusi Azimuthal Gap

In [ ]:
df_unique.source_gap_deg.describe()

In [ ]:
fig = ff.create_distplot([df_unique['source_gap_deg']], ['Earthquake Azimuthal Gap'])
fig.show()

## Jumlah Gempabumi Harian

In [ ]:
df.date_

In [ ]:
def day(df):
  for index, row in df.iterrows():
    datem = datetime.datetime.strptime(row['date_'],"%d/%m/%Y")
    df.at[index, 'day'] = datem.strftime("%d")
  return df

In [ ]:
df_day = day(df_unique)

In [ ]:
fig = px.histogram(df_day, x="day")
fig.show()

## Plot Gempa dan Stasiun


In [ ]:
df_sta = df.drop_duplicates(subset=['receiver_code'])

In [ ]:
min_lat, max_lat = -11, 8
min_lon, max_lon = 94, 142

fig = plt.gcf()
fig.set_size_inches(8, 6.5)

m = Basemap(projection='merc', \
            llcrnrlat=min_lat, urcrnrlat=max_lat, \
            llcrnrlon=min_lon, urcrnrlon=max_lon, \
            lat_ts=20, \
            resolution='c')

# m.drawmapboundary(fill_color='aqua')
m.bluemarble(scale=0.5)
# m.fillcontinents(color='#cc9955', lake_color='aqua', zorder = 0)
m.drawcoastlines(linewidth=0.5, color = 'yellow')
m.drawparallels(np.arange(min_lat,max_lat,3),labels=[1,0,0,0], linewidth=0.5)
m.drawmeridians(np.arange(min_lon,max_lon,10),labels=[0,0,0,1], linewidth=0.5)


x_eq, y_eq = m(df_unique.source_longitude.to_list(),df_unique.source_latitude.to_list())
plt.scatter(x_eq, y_eq, 10, marker='o', color='Red')

x_sta, y_sta = m(df_sta.receiver_longitude.to_list(),df_sta.receiver_latitude.to_list())
plt.scatter(x_sta, y_sta, 10, marker='^', color='orange')

plt.show()


# **3. Cleaning dan Filtering**

## Hilangkan data dari stasiun luar Indonesia

In [ ]:
df

In [ ]:
df.network_code.unique()

In [ ]:
df.source_id.value_counts()

In [ ]:
df_filter_network = df[df.network_code == "IA"]
df_filter_network

In [ ]:
df_filter_network.network_code.unique()

## Seleksi data berdasarkan koordinat

In [ ]:
df_unique_latitude = df_unique.query("-11 <= source_latitude <= -5")

In [ ]:
min_lat, max_lat = -11, 8
min_lon, max_lon = 94, 142

fig = plt.gcf()
fig.set_size_inches(8, 6.5)

m = Basemap(projection='merc', \
            llcrnrlat=min_lat, urcrnrlat=max_lat, \
            llcrnrlon=min_lon, urcrnrlon=max_lon, \
            lat_ts=20, \
            resolution='c')

# m.drawmapboundary(fill_color='aqua')
m.bluemarble(scale=0.5)
# m.fillcontinents(color='#cc9955', lake_color='aqua', zorder = 0)
m.drawcoastlines(linewidth=0.5, color = 'yellow')
m.drawparallels(np.arange(min_lat,max_lat,3),labels=[1,0,0,0], linewidth=0.5)
m.drawmeridians(np.arange(min_lon,max_lon,10),labels=[0,0,0,1], linewidth=0.5)


x_eq, y_eq = m(df_unique_latitude.source_longitude.to_list(),df_unique_latitude.source_latitude.to_list())
plt.scatter(x_eq, y_eq, 10, marker='o', color='Red')

# x_sta, y_sta = m(df_sta.receiver_longitude.to_list(),df_sta.receiver_latitude.to_list())
# plt.scatter(x_sta, y_sta, 10, marker='^', color='orange')

plt.show()

In [ ]:
df_unique_longitude = df_unique.query("94 <= source_longitude <= 105")

In [ ]:
min_lat, max_lat = -11, 8
min_lon, max_lon = 94, 142

fig = plt.gcf()
fig.set_size_inches(8, 6.5)

m = Basemap(projection='merc', \
            llcrnrlat=min_lat, urcrnrlat=max_lat, \
            llcrnrlon=min_lon, urcrnrlon=max_lon, \
            lat_ts=20, \
            resolution='c')

# m.drawmapboundary(fill_color='aqua')
m.bluemarble(scale=0.5)
# m.fillcontinents(color='#cc9955', lake_color='aqua', zorder = 0)
m.drawcoastlines(linewidth=0.5, color = 'yellow')
m.drawparallels(np.arange(min_lat,max_lat,3),labels=[1,0,0,0], linewidth=0.5)
m.drawmeridians(np.arange(min_lon,max_lon,10),labels=[0,0,0,1], linewidth=0.5)


x_eq, y_eq = m(df_unique_longitude.source_longitude.to_list(),df_unique_longitude.source_latitude.to_list())
plt.scatter(x_eq, y_eq, 10, marker='o', color='Red')

# x_sta, y_sta = m(df_sta.receiver_longitude.to_list(),df_sta.receiver_latitude.to_list())
# plt.scatter(x_sta, y_sta, 10, marker='^', color='orange')

plt.show()

In [ ]:
df_unique_jawa = df_unique.query("(104 <= source_longitude <= 115) and (-11 <= source_latitude <= -5)")

In [ ]:
min_lat, max_lat = -11, 8
min_lon, max_lon = 94, 142

fig = plt.gcf()
fig.set_size_inches(8, 6.5)

m = Basemap(projection='merc', \
            llcrnrlat=min_lat, urcrnrlat=max_lat, \
            llcrnrlon=min_lon, urcrnrlon=max_lon, \
            lat_ts=20, \
            resolution='c')

# m.drawmapboundary(fill_color='aqua')
m.bluemarble(scale=0.5)
# m.fillcontinents(color='#cc9955', lake_color='aqua', zorder = 0)
m.drawcoastlines(linewidth=0.5, color = 'yellow')
m.drawparallels(np.arange(min_lat,max_lat,3),labels=[1,0,0,0], linewidth=0.5)
m.drawmeridians(np.arange(min_lon,max_lon,10),labels=[0,0,0,1], linewidth=0.5)


x_eq, y_eq = m(df_unique_jawa.source_longitude.to_list(),df_unique_jawa.source_latitude.to_list())
plt.scatter(x_eq, y_eq, 10, marker='o', color='Red')

# x_sta, y_sta = m(df_sta.receiver_longitude.to_list(),df_sta.receiver_latitude.to_list())
# plt.scatter(x_sta, y_sta, 10, marker='^', color='orange')

plt.show()

## Seleksi data berdasarkan magnitude

In [ ]:
df_unique_magnitude = df_unique.query("source_magnitude >= 5")

In [ ]:
min_lat, max_lat = -11, 8
min_lon, max_lon = 94, 142

fig = plt.gcf()
fig.set_size_inches(8, 6.5)

m = Basemap(projection='merc', \
            llcrnrlat=min_lat, urcrnrlat=max_lat, \
            llcrnrlon=min_lon, urcrnrlon=max_lon, \
            lat_ts=20, \
            resolution='c')

# m.drawmapboundary(fill_color='aqua')
m.bluemarble(scale=0.5)
# m.fillcontinents(color='#cc9955', lake_color='aqua', zorder = 0)
m.drawcoastlines(linewidth=0.5, color = 'yellow')
m.drawparallels(np.arange(min_lat,max_lat,3),labels=[1,0,0,0], linewidth=0.5)
m.drawmeridians(np.arange(min_lon,max_lon,10),labels=[0,0,0,1], linewidth=0.5)


x_eq, y_eq = m(df_unique_magnitude.source_longitude.to_list(),df_unique_magnitude.source_latitude.to_list())
plt.scatter(x_eq, y_eq, 10, marker='o', color='Red')

# x_sta, y_sta = m(df_sta.receiver_longitude.to_list(),df_sta.receiver_latitude.to_list())
# plt.scatter(x_sta, y_sta, 10, marker='^', color='orange')

plt.show()

In [ ]:
df_unique_magnitude[['source_id','date_','source_latitude','source_longitude', 'source_depth_km', 'source_magnitude']]

## Input data tambahan pada kolom kosong

In [ ]:
df_store = df[df.network_code == "GE"]
df_store

In [ ]:
df_iris_station = pd.read_csv("wilber-stations.txt", sep="|")

In [ ]:
for index, row in df_store.iterrows():
  if row['receiver_code'] in df_iris_station['Station'].to_list():
    df_store.at[index,"receiver_latitude"] = df_iris_station[df_iris_station["Station"] == row['receiver_code']]['Latitude'].values[0]
    df_store.at[index,"receiver_longitude"] = df_iris_station[df_iris_station["Station"] == row['receiver_code']]['Longitude'].values[0]
    df_store.at[index,"receiver_elevation_m"] = df_iris_station[df_iris_station["Station"] == row['receiver_code']]['Elevation'].values[0]
  else:
    pass

In [ ]:
df_store.columns

# **4. Data Storing**

In [ ]:
df_store.reset_index(drop=True, inplace=True)

In [ ]:
df_store.to_csv("arrival_GE.csv", index=False)

# **5. Download Waveform**

In [ ]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

In [ ]:
# download data FDSN lain dengan spesifikasi username dan password
# url_ = str(url) # url FDSN
# usr_ = str(user) # username FDSN account
# pwd_ = str(pawd) # password FDSN account
# client = Client(url_,user=usr_,password=pwd_)

# download data via IRIS
client = Client("IRIS")

i = 0
t = UTCDateTime(df_store.source_origin_time[i])
st = client.get_waveforms(df_store.network_code[i],
                          df_store.receiver_code[i],
                          "*",
                          "BH?",
                          t,
                          t + 360)

inv = client.get_stations(network=df_store['network_code'][i],
                          station=df_store['receiver_code'][i],
                          starttime=t,
                          endtime=t + 360,
                          loc="",
                          channel="BH?",
                          level="response")
st.plot();

In [ ]:
pre_filt = [0.001, 0.005, 45, 50]
st = st.remove_response(inventory=inv,
                        pre_filt=pre_filt,
                        water_level=60, plot=False)

# st.plot();

In [ ]:
st.write(f"nama_file.mseed",
         format="MSEED")

In [ ]:
st.write(f"{df_store.network_code[i]}.{df_store.receiver_code[i]}.{df_store.date_[i].replace('/','')}.mseed",
         format="MSEED")